# Supplement Figures: S7

end of 27_summary-figs.ipynb

## S7: Decomposition of the change in near-surface relative humidity into temperature and specific humidity contributions

In [ ]:
norm_zavg_cesm_diff = {}
norm_zavg_hadhst_diff = {}
norm_zavg_hada1b_diff = {}
norm_zavg_cmip_diff = {}

norm_zavg_cesm_rhpart = {}

for d in ['all', 'lnd', 'ocn']:
    norm_zavg_cesm_diff[d] = {}
    norm_zavg_hadhst_diff[d] = {}
    norm_zavg_hada1b_diff[d] = {}
    norm_zavg_cmip_diff[d] = {}

    for v in ppe_varlist:
        norm_zavg_cesm_diff[d][v] = xr.full_like(zavg_cesm_diff[d][v], np.nan).transpose('member', 'lat')
        norm_zavg_hadhst_diff[d][v] = xr.full_like(zavg_hadhst_diff[d][v], np.nan).transpose('member', 'lat')
        norm_zavg_hada1b_diff[d][v] = xr.full_like(zavg_hada1b_diff[d][v], np.nan).transpose('member', 'lat')
    



for d in ['all', 'lnd', 'ocn']:
    norm_zavg_cesm_rhpart[d] = {}
    for v in ['dRH', 'dRH_q', 'dRH_T']:    
        norm_zavg_cesm_rhpart[d][v] = xr.full_like(zavg_cesm_rhpart[d][v], np.nan).transpose('member', 'lat')

for d in ['all', 'lnd', 'ocn']:
    for m in range(36):
        # Normalize dRH_q and dRH_T together
        member_subset = [
            zavg_cesm_rhpart[d]['dRH_q'].isel(member=m)/savg_cesm_diff['all']['global']['T_S'].isel(member=m),
            zavg_cesm_rhpart[d]['dRH_T'].isel(member=m)/savg_cesm_diff['all']['global']['T_S'].isel(member=m),
        ]
        norm_member_subset = normalize_to_centered_range(member_subset, scale=1)
        norm_zavg_cesm_rhpart[d]['dRH_q'][m] = norm_member_subset[0]
        norm_zavg_cesm_rhpart[d]['dRH_T'][m] = norm_member_subset[1]

        # Normalize dRH
        norm_zavg_cesm_rhpart[d]['dRH'] = normalize_to_centered_range(
            zavg_cesm_rhpart[d]['dRH'].isel(member=m)/savg_cesm_diff['all']['global']['T_S'].isel(member=m),
            scale=1
        )

In [ ]:
fig, axes = plt.subplots(
    1, 3, figsize=(12,3),
    layout='constrained',
    sharex=True,
    sharey=True,
)
ax = axes.ravel()

for i, d in enumerate(['all', 'lnd', 'ocn']):
    norm_zavg_cesm_rhpart[d]['dRH_q'].mean(dim='member').plot(ax=ax[i], c='b', lw=1.5, alpha=1, label='$\Vert \Delta RH_{q} \Vert$')
    norm_zavg_cesm_rhpart[d]['dRH_T'].mean(dim='member').plot(ax=ax[i], c='r', lw=1.5, alpha=1, label='$\Vert \Delta RH_{T} \Vert$')
    (norm_zavg_cesm_rhpart[d]['dRH_q'] + norm_zavg_cesm_rhpart[d]['dRH_T']).mean(dim='member').plot(ax=ax[i], c='k', lw=1.5, alpha=1, label='$\Vert \Delta RH_{q} \Vert$ + $\Vert \Delta RH_{T} \Vert$')

    ci95_q = stats.ttest_1samp(norm_zavg_cesm_rhpart[d]['dRH_q'], 0, axis=0, nan_policy='propagate').confidence_interval(confidence_level=0.95)
    ax[i].fill_between(
        norm_zavg_cesm_rhpart[d]['dRH_q'].lat, ci95_q[0], ci95_q[1],
        alpha=0.2, color='b', lw=0, zorder=0,
    )

    ci95_T = stats.ttest_1samp(norm_zavg_cesm_rhpart[d]['dRH_T'], 0, axis=0, nan_policy='propagate').confidence_interval(confidence_level=0.95)
    ax[i].fill_between(
        norm_zavg_cesm_rhpart[d]['dRH_T'].lat, ci95_T[0], ci95_T[1],
        alpha=0.2, color='r', lw=0, zorder=0,
    )

    ci95 = stats.ttest_1samp(norm_zavg_cesm_rhpart[d]['dRH_q'] + norm_zavg_cesm_rhpart[d]['dRH_T'], 0, axis=0, nan_policy='propagate').confidence_interval(confidence_level=0.95)
    ax[i].fill_between(
        norm_zavg_cesm_rhpart[d]['dRH_T'].lat, ci95[0], ci95[1],
        alpha=0.2, color='k', lw=0, zorder=0,
    )

    ax[i].set_xticks([-30, -15, 0, 15, 30])
    ax[i].set_yticks([-1, -0.5, 0, 0.5, 1])
    ax[i].grid(lw=0.75, ls=':', c='k', zorder=0)
    ax[i].axhline(lw=0.75, ls='-', c='k', zorder=1)
    ax[i].set_ylim(-1, 1)
    ax[i].set_xlim(-30, 30)
    ax[i].set_xlabel('Latitude [$^{\circ}$N]')
    # ax[i].set_ylabel('$\Vert \Delta RH \Vert$')


ax[0].set_ylabel('$\Vert \Delta RH \Vert$')

ax[0].set_title(f'Land & Ocean', fontweight='bold', fontsize=12)
ax[1].set_title(f'Land', fontweight='bold', fontsize=12)
ax[2].set_title(f'Ocean', fontweight='bold', fontsize=12)

ax[1].legend(loc='upper left', fontsize=8)

plt.savefig('./ms_figs_v2/pdf/zmrh_part_norm.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.savefig('./ms_figs_v2/png/zmrh_part_norm.png', format='png', dpi=300, bbox_inches='tight')